In [123]:
import pandas as pd
import numpy as np
import time
import math
import sklearn.model_selection
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn import preprocessing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [140]:
df = pd.read_csv('IMBD_train.csv')

In [212]:
df = pd.read_csv('feature_eng_v1.csv')

In [254]:
df = pd.read_csv('feature_eng_v2.csv')

In [255]:
df.iloc[:,1] = df.iloc[:,1].str.replace('.csv', '')
df = df.drop(df.columns[0], axis=1)

In [256]:
for index, row in df.iterrows():
    for j in range(5,28):
        if row[j] == 0 or row[j] == -1:
            if j == 5:
                #replace with col 6's val
                new_val = df.iloc[index,j+1]
                df.iloc[index,j] = new_val
            elif j == 28:
                #replace with col 27's val
                new_val = df.iloc[index,j-1]
                df.iloc[index,j] = new_val
            else:
                #replace with 前後
                new_val = (df.iloc[index,j+1] + df.iloc[index,j-1])/2
                df.iloc[index,j] = new_val

In [292]:
feature = []
for index, row in df.iterrows():
    new_row = []
    new_row.append(list(df.iloc[index,5:5+19]))
    new_row.append(list(df.iloc[index,30:30+19]))
    new_row.append(list(df.iloc[index,52:52+19]))
    new_row.append(list(df.iloc[index,74:74+19]))
    new_row.append(list(df.iloc[index,94:94+19]))
    new_row.append(list(df.iloc[index,115:115+19]))
    new_row.append(list(df.iloc[index,134:134+19]))
    new_row.append(list(df.iloc[index,157:157+19]))
    new_row.append(list(df.iloc[index,179:179+19]))
    new_row.append(list(df.iloc[index,200:200+19]))
    new_row.append(list(df.iloc[index,220:220+19]))
    new_row.append(list(df.iloc[index,242:242+19]))
    new_row.append(list(df.iloc[index,263:263+19]))
    new_row.append(list(df.iloc[index,283:283+19]))
    feature.append(new_row)
    feature_df = pd.DataFrame({'new_features':feature})
new_df = pd.concat([df.iloc[:,:5],feature_df],axis=1)
new_df.head(2)

,1,2,3,4,5,new_features
0,A11887,G21_1230,20200219,PTC13,Deg.F,"[[66.7, 66.7, 66.7, 66.7, 69.9, 71.7, 73.0, 74..."
1,A11887,G21_1230,20200219,PTC14,Deg.F,"[[66.7, 66.7, 66.7, 66.7, 68.2, 69.1, 70.0, 71..."


In [150]:
# df_copy = df.iloc[:,5:].copy()
# norm_df.iloc[:,:] = min_max_scaler.fit_transform(df_copy)
# norm_df = pd.concat([df.iloc[:,:5],norm_df],axis=1)
# norm_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,A11887,G21_1230,20200219,PTC13,Deg.F,0.256024,0.019345,0.019355,0.252079,0.021083,...,0.360439,0.371785,0.386717,0.395613,0.400984,0.414523,0.430577,0.445247,0.462738,0.478460
1,A11887,G21_1230,20200219,PTC14,Deg.F,0.256024,0.019345,0.019355,0.252079,0.014904,...,0.321861,0.332073,0.344213,0.351362,0.355547,0.366868,0.380121,0.394677,0.406844,0.422417


In [393]:
# for col in df.columns.values:
#     df = df[df[col] != -1]

In [31]:
# workpiece = df['0'].unique()
# group = df['1'].unique()
# print(len(workpiece),workpiece, '\n', len(group), group)

In [32]:
count_w = df.groupby('0').count().iloc[:,0].values
count_g = df.groupby('1').count().iloc[:,0].values
print(count_w,'\n',count_g)

[108 147  57  15  12  39  21  12  12 108   8  20 183  39 131  21  26  21
 149  32  69  17  19 163  20   9  22  22  96 144 147  72] 
 [426 129 230 471 175 379 151]


In [293]:
train_df, val_df = train_test_split(new_df, test_size=0.1)
print(len(train_df),len(val_df))

1764 197


In [294]:
weight_w = len(train_df)/count_w
weight_g = len(train_df)/count_g
print(weight_w,'\n',weight_g)

[ 16.33333333  12.          30.94736842 117.6        147.
  45.23076923  84.         147.         147.          16.33333333
 220.5         88.2          9.63934426  45.23076923  13.46564885
  84.          67.84615385  84.          11.83892617  55.125
  25.56521739 103.76470588  92.84210526  10.82208589  88.2
 196.          80.18181818  80.18181818  18.375       12.25
  12.          24.5       ] 
 [ 4.14084507 13.6744186   7.66956522  3.74522293 10.08        4.65435356
 11.68211921]


In [34]:
# #sliding window:
# sw_df = df.iloc[:,:5].copy()
# for i in range(5,22,2):
#     #print(i)
#     col = list(df.iloc[:,i:i+6].values)
#     #print(col,len(col))
#     sw_df[str(i)] = col
# sw_df.head(3)

# train_df, val_df = train_test_split(sw_df, test_size=0.2)
# print(len(train_df),len(val_df))

In [307]:
>>> m = nn.MaxPool1d(3, stride=2)
>>> input = torch.randn(20, 16, 50)
>>> output = m(input)
output.shape

torch.Size([20, 16, 24])

In [217]:
class Dataset(Dataset):
    def __init__(self, mode, df):
        assert mode in ["train", "test"]  
        self.mode = mode
        self.index = df.iloc[:,0]
        self.data = df
        self.len = len(self.data)

    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        x = self.data.iloc[idx,5:]
        if self.mode == "train":
            workpiece = list(df['1'].unique())
            group = list(df['2'].unique())
            w_name = self.data.iloc[idx][0]
            g_name = self.data.iloc[idx][1]
            
            w_index = workpiece.index(w_name)
            g_index = group.index(g_name)
            input_x = torch.tensor(x)
            
            return input_x.float(), torch.tensor([w_index]).float(), torch.tensor([g_index]).float()
        else:
            return torch.tensor(x).float()

In [310]:
class CNN_Model(nn.Module):
    def __init__(self, in_channel, out_channel, input_size, output_dim_g, output_dim_w, batch_size):
        super().__init__()
        self.cnn1 = nn.Conv1d(in_channel, out_channel, kernel_size=3)
        self.cnn2 = nn.Conv1d(out_channel, out_channel, kernel_size=9)
        self.relu1 = nn.ReLU(inplace=True) 
        self.pool = nn.MaxPool1d(3, stride=2)
        self.fc_g = nn.Linear(19*6, output_dim_g)
        self.fc_w = nn.Linear(19*6, output_dim_w)
        self.flatten = nn.Flatten()
        self.bs = batch_size
        
    def forward(self, x):
        out = self.cnn1(x)
        out = self.pool(x)
        out = self.relu1(out)
        #print(out.shape)
        #out = self.cnn2(out)
        #out = self.relu1(out)
        #print(out.shape)
        out = self.flatten(out)
        
#         out_g = self.fc_g(out)
#         out_w = self.fc_w(out)
        out_w = self.fc_w(out) # 64 * input_dim(23 for raw or 258) * 1
        out_g = []
        for batch in range(out_w.shape[0]):
            val1, _ = out_w[batch][:3].topk(1) 
            val2, _ = out_w[batch][3:6].topk(1)
            val3, _ = out_w[batch][6:13].topk(1) #7
            val4, _ = out_w[batch][13:20].topk(1) #7
            val5, _ = out_w[batch][20:25].topk(1)
            val6, _ = out_w[batch][25:29].topk(1)
            val7, _ = out_w[batch][29:].topk(1)
            out_g.append([val1,val2,val3,val4,val5,val6,val7])
        out_g = torch.tensor(out_g).cuda()
        return out_w.squeeze(1), out_g.squeeze(1)
        

In [246]:
m = nn.Conv2d(16, 33, 3, stride=2)
input = torch.randn(20, 16, 50, 100)
output = m(input)
output.shape

torch.Size([20, 33, 24, 49])

In [210]:
>>> m = nn.Conv1d(12, 32, 5, stride=1)
>>> input = torch.randn(2, 12, 19)
>>> output = m(input)

In [208]:
output.shape

torch.Size([2, 32, 15])

In [68]:
>>> m = nn.Conv1d(32, 20, 11, stride=1)
>>> input = torch.randn(20, 32, 19)
>>> output = m(input)
output.shape

torch.Size([20, 20, 9])

In [61]:
torch.flatten(output,start_dim=1).shape

torch.Size([20, 608])

In [27]:
output.view(20,-1).shape

torch.Size([20, 608])

In [189]:
def calc_score(accuracy):
    if accuracy >= .75:
        return 100
    elif accuracy >= .65:
        return 90
    elif accuracy >= .55:
        return 80
    elif accuracy >= .45:
        return 70
    elif accuracy >= .35:
        return 60
    else:
        return 0

In [312]:
def asMinutes(s): #s = time.time()-start_time
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def evaluation(valloader,net):
    net = net.eval()
    total, tp_w, tp_g, d = 0, 0, 0, 0
    with torch.no_grad():
        for x, true_w, true_g in valloader:
            #x = x.unsqueeze(1)
            x = x.squeeze(1)
            x = x.permute(0,2,1)
            x = x.cuda()
            true_w = true_w.cuda()
            true_g = true_g.cuda()
            pred_w, pred_g = net(x)
            if torch.argmax(pred_g,dim=1) == true_g:
                tp_g += 1
            if torch.argmax(pred_w,dim=1) == true_w:
                tp_w += 1
            if torch.argmax(pred_w,dim=1) == true_w and torch.argmax(pred_g,dim=1) == true_g:
                d += 1
            total += 1
    return (tp_w/total, tp_g/total, d/total)
    
def trainIter(model, trainloader, valloader, epochs, LR, weight_w, weight_g):
    model = model.cuda()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    loss_func_w = torch.nn.CrossEntropyLoss(weight=torch.FloatTensor(weight_w).cuda())
    loss_func_g = torch.nn.CrossEntropyLoss(weight=torch.FloatTensor(weight_g).cuda())
    start_time = time.time()
    best_score = 0
    for epoch in range(epochs):
        model = model.train()
        steps, train_loss, total_wloss, total_gloss = 0, 0, 0, 0
        for x, y_w, y_g in trainloader:
            #x = x.unsqueeze(1)
            x = x.squeeze(1)
            x = x.permute(0,2,1)
            x = x.cuda()
            y_w = y_w.cuda()
            y_g = y_g.cuda()
            pred_w, pred_g = model(x)
            loss_w = loss_func_w(pred_w, y_w.squeeze(1).long())
            loss_g = loss_func_g(pred_g, y_g.squeeze(1).long())
            final_loss = loss_g*0.5 + loss_w*0.5
            final_loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            train_loss += final_loss.item()
            total_wloss += loss_w.item()
            total_gloss += loss_g.item()
            steps += 1
            print(f'Epoch : {epoch+1}/{epochs}, time: {asMinutes(time.time()-start_time)}, Training Loss : {train_loss/steps},Group Loss : {total_gloss/steps},WorkPiece Loss : {total_wloss/steps}', end = '\r')
        tp_w, tp_g, d = evaluation(valloader, model)
        score = calc_score(d)*0.3 + calc_score(tp_g)*0.7
        if score >= best_score:
            torch.save(model.state_dict(), f'IMBD_final_1013_cnn') 
            best_score = score
    
        print(f'Epoch : {epoch+1}/{epochs}, Accuracy : {score}/{tp_g}/{tp_w}/{d}',  end = '\r') 
        if epoch % 50 == 0:
            print('\n===============================================================================================')

In [193]:
train = Dataset('train', train_df)
trainloader = torch.utils.data.DataLoader(train, batch_size= 262, drop_last = True,shuffle=True)
for x,y_w, y_g in trainloader:
    print(x.shape, y_w.shape)
#     print(x)
    break

torch.Size([262, 258]) torch.Size([262, 1])


In [314]:
input_size = 19
HIDDEN = 256
N = 1
BS = 32
out_g = 7
out_w = 32 
dropout = 0.3
#model = classifier(output_dim_g=output_dim_g, output_dim_w=output_dim_w, input_size=input_size, hidden_dim=HIDDEN, n_layers=N, batch_size=BS, drop_prob=dropout)
model = CNN_Model(in_channel=19, out_channel=10, input_size=input_size, output_dim_g= out_g, output_dim_w=out_w, batch_size=BS)
train = Dataset('train', train_df)
trainloader = torch.utils.data.DataLoader(train, batch_size= BS, drop_last = True,shuffle=True)
val = Dataset('train', val_df)
valloader = torch.utils.data.DataLoader(val, batch_size=1, shuffle=False)
trainIter(model, trainloader, valloader, epochs=700, LR=2e-4, weight_w=weight_w, weight_g=weight_g)

Epoch : 1/700, Accuracy : 0.0/0.2233502538071066/0.08629441624365482/0.0862944162436548207794,WorkPiece Loss : 46.4452060872858256
Epoch : 51/700, Accuracy : 49.0/0.5076142131979695/0.2436548223350254/0.243654822335025411704878,WorkPiece Loss : 1.9309954209761186
Epoch : 101/700, Accuracy : 56.0/0.6345177664974619/0.2639593908629442/0.263959390862944275844782,WorkPiece Loss : 1.7838089617815884
Epoch : 151/700, Accuracy : 56.0/0.6395939086294417/0.28426395939086296/0.2842639593908629668908,WorkPiece Loss : 1.67456556883725254
Epoch : 201/700, Accuracy : 56.0/0.6345177664974619/0.25380710659898476/0.2538071065989847619275,WorkPiece Loss : 1.70113610679453085
Epoch : 251/700, Accuracy : 56.0/0.6040609137055838/0.29441624365482233/0.2944162436548223349401,WorkPiece Loss : 1.66817940365184454


KeyboardInterrupt: 

In [317]:
tp_w, tp_g, d = evaluation(valloader, model)
score = calc_score(d)*0.3 + calc_score(tp_g)*0.7
print(score,tp_w,tp_g,d)

88.0 0.4071246819338422 0.7608142493638677 0.3638676844783715


In [ ]:
print(1)